## Usando arquivo .env para controlar variaveis de ambiente
Para evitar exposição da chave `OPENAI_API_KEY` optei por utilizar arquivo `.env` com a informação da chave.

Para seguir o mesmo método basta criar um arquivo `.env` no mesmo diretório do arquivo `my-first-crew.ipynb`.
A importação da chave será feita através da célula abaixo que faz a instalação de um biblioteca para carregar
os valores do arquivo `.env`.

In [17]:
%%capture
%pip install python-dotenv
import dotenv
%load_ext dotenv
%dotenv

In [18]:
%%capture
!pip install crewai==0.28.8 crewai-tools==0.1.6 langchain-community==0.0.29

In [19]:
from crewai import Agent, Task, Crew


In [20]:
import os
os.environ['OPENAI_MODEL_NAME'] = "gpt-3.5-turbo"

# Grupo de agente para escrever artigos

1. Planner - Planejador do artigo
2. Writer - Escrito do artigo
3. Editor - Editor do artigo

In [21]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging content about the {topic}",
    backstory="""You're working on planning a blog post article about
    the topic: {topic}.
    You collect information that helps the audience learn something
    and mak informed decisions.
    Your work is the basics for the Content Writer to write article on this topic.
    """,
    allow_delegation=False,
    verbose=True,
)

In [22]:
writer = Agent(
    role="Content Writer",
    goal="Write insightfull and factually accurate opinion piece about the topic: {topic}",
    backstory="""You're working on writing and opinion about the topic: {topic}.
    You base your opinion on the work of the Conntent Planner, who provides an outline and relevant context
    about the topic. You follow the main objectives and direction of the outline, as provided by the Content Planner.
    You also provide objective and impartial insights and back them with information provided the Content Planner.
    """,
    allow_delegation=False,
    verbose=True,
)

In [23]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post article to align with writing style of the organization.",
    backstory="""You're are a editor that receives a blog post from the Content Writer.
    Your goal is to review the blog post to ensure that it follow a journalistic best practices.
    """,
    allow_delegation=False,
    verbose=True,
)

In [24]:
plan = Task(
    description=(
        "1. Prioritaze the last trends, key players, and noteworthy news on {topic}"
        "2. Identify the target audience, considering their interests and pain points."
        "3. Develop a detaild content outline including an introduction, key points, and call to action."
        "4. Include SEO keyword and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document with an outline, audience analysis, SEO keywords and resources",
    agent=planner,
)

In [25]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling blog post on topic: {topic}."
        "2. Incorporate the SEO keyword naturally."
        "3. Sections/Subtitles are properly named in engaging manner."
        "4. Ensure the post is scructured with and engaging introduction, insightfull body and summarization conclusion."
    ),
    expected_output="A well-writen blog post in Markdown format, ready for publication, each section should have 2 or 3 paragraphs",
    agent=writer,
)

In [26]:
edit = Task(
    description="Proofread the given blog post for grammatical errors and aligment with the brand's voices.",
    expected_output="A well-writen blog post in Markdown format, ready for publication, each section should have 2 or 3 paragraphs",
    agent=editor,
)